In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt


In [2]:
# Define Constants
BATCH_SIZE = 32
IMAGE_SIZE = 224
CHANNELS = 3
EPOCHS = 20
DATASET_PATH = 'C:/Users/Aayushii Singh/Desktop/Adro/Data images'

In [ ]:
# Load Dataset from Directory
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    DATASET_PATH,
    seed=123,
    shuffle=True,
    batch_size=BATCH_SIZE,
    image_size=(IMAGE_SIZE, IMAGE_SIZE)
)

Found 2900 files belonging to 4 classes.


In [4]:
# Split Dataset into Train, Validation, and Test
def get_split_data(ds, train_split=0.8, test_split=0.1, val_split=0.1):
    assert (train_split + test_split + val_split) == 1
    ds_size = len(ds)
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds, val_ds, test_ds

train_ds, val_ds, test_ds = get_split_data(dataset)

In [5]:
# Prefetch and Cache Datasets for Performance Optimization
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

In [6]:
# Define Data Augmentation and Preprocessing
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.1),
])

In [7]:
# Build Custom CNN Model
input_shape = (IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
n_classes = len(dataset.class_names)

model = models.Sequential([
    data_augmentation,
    layers.Rescaling(1.0/255),
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(n_classes, activation='softmax')
])


In [8]:
# Compile the Model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])


In [9]:
# Train the Model
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS
)

Epoch 1/20
72/72 [==============================] - 92s 1s/step - loss: 0.8920 - accuracy: 0.5890 - val_loss: 0.4701 - val_accuracy: 0.7639
Epoch 2/20
72/72 [==============================] - 78s 1s/step - loss: 0.5137 - accuracy: 0.7461 - val_loss: 0.3799 - val_accuracy: 0.7812
Epoch 3/20
72/72 [==============================] - 77s 1s/step - loss: 0.4174 - accuracy: 0.7778 - val_loss: 0.3677 - val_accuracy: 0.7986
Epoch 4/20
72/72 [==============================] - 79s 1s/step - loss: 0.4213 - accuracy: 0.7812 - val_loss: 0.3442 - val_accuracy: 0.7951
Epoch 5/20
72/72 [==============================] - 79s 1s/step - loss: 0.4173 - accuracy: 0.7852 - val_loss: 0.3512 - val_accuracy: 0.7847
Epoch 6/20
72/72 [==============================] - 76s 1s/step - loss: 0.3714 - accuracy: 0.8108 - val_loss: 0.3537 - val_accuracy: 0.8090
Epoch 7/20
72/72 [==============================] - 76s 1s/step - loss: 0.3543 - accuracy: 0.8082 - val_loss: 0.3230 - val_accuracy: 0.7951
Epoch 8/20
72/72 [==

In [10]:
# Save the Model
model.save('drowsiness_model_3.h5')

In [22]:
# Real-Time Detection Using OpenCV with Custom Threshold Logic
def real_time_detection(model, class_names):
    # Load the pre-trained model
    cap = cv2.VideoCapture(0)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Preprocess Frame for Prediction
        img = cv2.resize(frame, (IMAGE_SIZE, IMAGE_SIZE))
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        img_array = tf.expand_dims(img_array, 0)  # Create batch axis
        
        # Make Prediction
        predictions = model.predict(img_array)
        confidence_scores = predictions[0]
        
        # Extract confidence for each class
        confidence_dict = {class_names[i]: confidence_scores[i] * 100 for i in range(len(class_names))}
        
        # Determine if the state is "Drowsy" or "Non-drowsy"
        is_drowsy = (
            # confidence_dict.get('eyes open', 0) < 50 or
            confidence_dict.get('eyes closed', 0) >= 0.15 or
            confidence_dict.get('yawn', 0) >= 35 
            # confidence_dict.get('no yawn', 0) < 50
        )
        
        status = "Non-Drowsy" if is_drowsy else "Drowsy"
        
        # Display Result
        cv2.putText(frame, f'{status}', (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0) if status == "Non-drowsy" else (0, 0, 255), 2)
        
        # Optionally, display confidence levels
        # y_offset = 60
        # for class_name, conf in confidence_dict.items():
        #     cv2.putText(frame, f'{class_name}: {conf:.2f}%', (10, y_offset),
        #                 cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 192, 203), 1)
        #     y_offset += 25
        
        cv2.imshow('Real-Time Detection', frame)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


In [24]:
# Load the Model and Run Real-Time Detection
model = tf.keras.models.load_model('drowsiness_model_3.h5')
class_names = dataset.class_names
real_time_detection(model, class_names)

1/1 [==============================] - 0s 67ms/step
